<a href="https://colab.research.google.com/github/gretiere545/corpus/blob/main/corpus_sync_airtable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corpus Airtable API

In [1]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
!pip install gspread --upgrade
# fix here => https://stackoverflow.com/questions/71347973/modulenotfounderror-no-module-named-gspread-models
!pip uninstall gspread-dataframe
!pip install gspread-dataframe
!pip install gspread-formatting
!pip install airtable-python-wrapper
import pandas as pd
import numpy as np
import requests
import json
import uuid
import datetime
pd.set_option("display.width",1000)
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA
from google.colab import files

# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting import *
from oauth2client.client import GoogleCredentials
# fix here => https://github.com/burnash/gspread/issues/1014
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
#gc = gspread.authorize(GoogleCredentials.get_application_default())
from airtable import Airtable
!export PYTHONIOENCODING=utf8


  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2
Found existing installation: gspread-dataframe 3.0.8
Uninstalling gspread-dataframe-3.0.8:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/gspread_dataframe-3.0.8.dist-info/*
    /usr/local/lib/python3.7/dist-packages/gspread_dataframe.py
Proceed (y/n)? y
  Successfully uninstalled gspread-dataframe-3.0.8
Mounted at /content/drive
/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


In [2]:
def get_cc_config(f):
  vk = []
  try:
    with open(f) as vk_f:
        vk = json.load(vk_f)
  except BaseException as e:
    print(e)
  return vk

def set_cc_config (vk, f):
  # sauvegarde du fichier de paramétrage
  with open(f, 'w') as fp:
      json.dump(vk, fp)  

In [3]:
#
# Ouverture de la Sheet Centrale (corpus_central_base)
#
def get_CCDB_wb(uri):
  sheet_central = uri
  wb_central = gc.open_by_url(sheet_central)
  return wb_central

def get_CCDB_data(wb, idx):
  t_corpus = wb.get_worksheet(idx)
  data_t_corpus = t_corpus.get_all_values()
  return data_t_corpus

#
# DataFrame CC global (toutes les langues)
#
def get_ccdf_global(CCDB):
  return pd.DataFrame(CCDB[1:], columns=CCDB[0])

#
# DataFrame CC Français (invariable)
#
def get_ccdf_fr(df_cc):
  # on supprime les doublons
  return df_cc[['uid','expression','theme','état','date','commentaires','index']]

In [4]:
# Config Langues
config_languages = 'med_vac_synthese.json'
vk_languages = get_cc_config(config_languages)

In [5]:
# Ouverture de la Sheet Centrale
sheet = 'https://docs.google.com/spreadsheets/d/1j9mtvKJn0Ad3FqxoJXMSKLJOfYTDOhtw3wxXZ4sQAIc'
wb  = gc.open_by_url(sheet)

In [6]:
# Config Corpus (de consolidation)
config_corpus_central = "corpus_central_config.json"
vk_corpus_central = get_cc_config(config_corpus_central)

In [7]:
# Base Centrale
# on charge l'onglet 1 (français)
db_cc_wb = get_CCDB_wb(vk_corpus_central[0]["uri"])
db_cc = get_CCDB_data (db_cc_wb, 1)
df_cc_global = get_ccdf_global(db_cc)
df_cc_fr = get_ccdf_fr(df_cc_global).drop_duplicates()

# Comparaison des versions de référentiels

In [8]:
def getLastModifiedTime (wb):
  # https://stackoverflow.com/questions/64254039/python-how-to-check-if-a-google-spreadsheets-was-updated
  # https://stackoverflow.com/questions/52260789/update-googlesheet-cell-with-timestamp-from-python
  revisions_uri = f'https://www.googleapis.com/drive/v3/files/{wb.id}/revisions'
  headers = {'Authorization': f'Bearer {GoogleCredentials.get_application_default().get_access_token().access_token}'}
  response = requests.get(revisions_uri, headers=headers).json()
  return response['revisions'][-1]['modifiedTime']

In [9]:
print (getLastModifiedTime(db_cc_wb))

2022-04-01T12:52:39.368Z


In [10]:
df_cc_global.loc[df_cc_global["expression"]=="Anesthésie générale"]

,uid,expression,date,theme,état,commentaires,index,ams,eng,tur,...,arm,dar,pst,prs,aze,esp,amh,all,pol,som
174,ef2f771b,Anesthésie générale,,Médecine générale,,,A,تخدير عام,,Genel anestezi,...,,,,,,,,,Znieczulenie ogólne,


### Appel de l'API
https://towardsdatascience.com/downloading-airtable-data-into-python-89e5c7107a24

In [11]:
params = ()
api_key = "keyrI98TIqu6mbFcf" 
api_key_gmail = "keysgpJWq3aWwqSZe"
headers = {"Authorization": "Bearer " + api_key,"Content-Type" : "application/json"}
base_id = "appawT3gg7cJhPYv0" # Base Nouvelle HV
table_name = 'Corpus'

In [12]:
airtable = Airtable(base_id, table_name, api_key)

```
for page in airtable.get_iter(view='Grid view',sort='Proposition'):
  for record in page:
    value = record['fields']['Proposition']
```



In [13]:
def getAllAirtableRecords():
  vk_at = airtable.get_all(view='Admin',sort='Proposition')
  df_vkat = pd.DataFrame.from_records((r['fields'] for r in vk_at))
  return df_vkat

In [14]:
def getNotInGSheet(df_cc_global, df_vkat):
  df_not_in_gsheet = df_vkat[~df_vkat["UID"].isin(df_cc_global["uid"])]
  return df_not_in_gsheet

In [15]:
df_vkat = getAllAirtableRecords()

## Synchronisation du serveur vers airtable

### 1. Différence entre A et B (sur UID)

In [16]:
df_not_in_airtable = df_cc_global[~df_cc_global["uid"].isin(df_vkat["UID"])].dropna()
df_not_in_airtable

,uid,expression,date,theme,état,commentaires,index,ams,eng,tur,...,arm,dar,pst,prs,aze,esp,amh,all,pol,som


### On ajoute les éléments de A absents dans B

In [17]:
for index, row in df_not_in_airtable.iterrows():
  if row["uid"] is not None:
    record = airtable.match('UID', row["uid"])
    if bool(record) is False:
      aDict = {}
      key = "UID"
      aDict[key] = row["uid"]
      aDict["Proposition"] = row["expression"]
      aDict["idx"] = row["index"]
      for i in vk_languages:
        key = i["trigramme"]
        aDict[key] = row[key]
      airtable.insert(aDict)

### On met à jour les traductions de A présents dans B

> Bloc en retrait



In [ ]:
cpt = 0
max = len(df_cc_global) 
for index, row in df_cc_global.iterrows():
   if row["uid"] is not None:
    record = airtable.match('UID', row["uid"])
    if bool(record):
      aDict = {}
      key = "UID"
      aDict[key] = row["uid"]
      for i in vk_languages:
        key = i["trigramme"]
        aDict[key] = row[key]
      airtable.update(record['id'], aDict)
      cpt+=1
      print(str(cpt/max) + " %", end="\r", flush=True)      
      #print (record['id'] +" " + row["uid"] +" " + row["tur"])

### On récupère les éléments de B absents de A

In [18]:
df_not_in_gsheet = getNotInGSheet(df_cc_global, df_vkat)

In [19]:
df_not_in_gsheet[["Proposition","idx_cor","UID"]]

,Proposition,idx_cor,UID
2,Abdomen,A,80cbacea
6,Accouchement,A,b67d8739
7,Accoucher,A,d1746bc6
8,Âge,A,f54f56c9
9,Alcool,A,57f5b4fb
...,...,...,...
321,Urine,U,0e25b5fb
323,Vaccinal,V,6c70b10f
325,"Vacciné, -e",V,571d5aa0
334,Votre poids a-t-il augmenté ?,P,8e8c22aa


## Mise en forme et ajustements dans Airtable

In [20]:
def make_unique_id():
  return str(uuid.uuid4())[:8]

In [21]:
df_vkat = getAllAirtableRecords()
df_not_in_gsheet = getNotInGSheet(df_cc_global, df_vkat)

### Update des UIDS manquants

In [22]:
vk_uid = df_vkat['UID'].tolist()
for page in airtable.get_iter(view='Admin',sort='Proposition'):
  for record in page:
    try:
      value = record['fields']['UID']
    except KeyError:
      print ("mssing uid")
      while True:
        auid = make_unique_id()
        if auid not in vk_uid:
          aDict = {}
          aDict["UID"] = auid
          print (record['id'] + " " + aDict["UID"])
          airtable.update(record['id'], aDict)
          vk_uid.append(auid) 
          break     


## Toggle des enregistrements de AT à ajouter dans la sheet locale

In [ ]:
df_vkat = getAllAirtableRecords()
df_not_in_gsheet = getNotInGSheet(df_cc_global, df_vkat)
# on met à jour le status dans Airtable
for index, row in df_not_in_gsheet.iterrows():
  if row["UID"] is not None:
    record = airtable.match('UID', row["UID"])
    if bool(record) is True:
      aDict = {}
      aDict["AppendToCorpus"] = True
      airtable.update(record['id'], aDict)
      print ("Updating status for " + row["UID"])

# Report dans la sheet locale

In [ ]:

# Ouverture de la Sheet Locale (mots à traduire)
sheet = 'https://docs.google.com/spreadsheets/d/1M3CFSpAGII25qCl0apYrRPX-DmpdxDFLLB_Ob-MX8DQ/'
wb  = gc.open_by_url(sheet)
t_corpus = wb.worksheet("fra_med_gen")
data_t_corpus = t_corpus.get_all_values()
df_local = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0]).drop_duplicates()  
df_not_in_gsheet.rename(columns={'UID': 'uid', 'Proposition': 'français'}, inplace=True)
df_not_in_gsheet = df_not_in_gsheet[["uid","français"]]
frames = [df_local, df_not_in_gsheet]
result = pd.concat(frames)
result.drop_duplicates(subset=['uid'], inplace=True, keep='last')
set_with_dataframe(t_corpus, result)

# Regroupement pour Langdeck

In [23]:
df_vkat = getAllAirtableRecords()
df_langdeck = df_vkat.copy()
df_langdeck = df_langdeck[["UID","Maitre","Ordre","Proposition","idx_cor","lastModified","ams","eng","tur","rus","ukr","rou","hun","alb","geo","arm","dar","pst","prs","esp","pol","tig"]]
df_langdeck.rename(columns={"UID":"uid","Proposition":"expression","idx_cor":"idx", "Maitre":"maitre","Ordre":"ordre","lastModified":"timestamp"}, inplace=True)
# https://stackoverflow.com/questions/28654047/convert-columns-into-rows-with-pandas
df2 = pd.melt(df_langdeck, id_vars=["uid", "maitre","ordre","expression","idx","timestamp"], 
                  var_name="language", value_name="translation")

In [34]:
#https://stackoverflow.com/questions/60393668/pandas-generate-unique-id-based-on-row-values
df2['hash'] = df2[['uid', 'language']].sum(axis=1).map(hash)

In [35]:
df_langdeck[df_langdeck['uid'].duplicated()].sort_values("uid")

,uid,maitre,ordre,expression,idx,timestamp,ams,eng,tur,rus,...,hun,alb,geo,arm,dar,pst,prs,esp,pol,tig


### Vérification qu'il n'existe pas des doublons

In [36]:
df2[df2['hash'].duplicated()].sort_values("hash")

,uid,maitre,ordre,expression,idx,timestamp,language,translation,hash,language_fulltext


#### Fonction lambda pour récupérer le nom long de la langue
https://stackoverflow.com/questions/2191699/find-an-element-in-a-list-of-tuples

In [37]:
df2["language_fulltext"] = df2["language"].apply(lambda x:[item for item in [(a_dict["trigramme"],a_dict["language"]) for a_dict in vk_languages] if item[0] == x][0][1])

#### Ajout du timestamp

In [ ]:
def append_timestamp():
     ct = datetime.datetime.now().isoformat()
     return ct

In [38]:
df2['timestamp'] = df2['timestamp'].astype('datetime64[s]')

In [29]:
len(df2)

5456

In [30]:
df2.dtypes

uid                          object
maitre                       object
ordre                         int64
expression                   object
idx                          object
timestamp            datetime64[ns]
language                     object
translation                  object
hash                          int64
language_fulltext            object
dtype: object

In [31]:
# Ouverture de la Sheet Langdeck (mots à traduire)

#sheet = 'https://docs.google.com/spreadsheets/d/1cSnSnnQunL-I7hpyYkIqbtuGr55unVJ3Qe8URSrjTr8/'
#t_corpus = wb.worksheet("Langdeck")
sheet = 'https://docs.google.com/spreadsheets/d/1bkL0bK2yh6Ea_BQaFIJ_yAvbTXw8NOFtlTZJSZK_5FY/'
wb  = gc.open_by_url(sheet)
t_corpus = wb.worksheet("t_sync_voc")
data_t_corpus = t_corpus.get_all_values()
df_local = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0]).drop_duplicates()  
df_local['hash'] = pd.to_numeric(df_local['hash'])
df_local['ordre'] = pd.to_numeric(df_local['ordre'])
df_local['timestamp'] = df_local['timestamp'].astype('datetime64[s]')

In [85]:
df2['hash'] = df2['hash'].astype(str)

In [86]:
l = len(df_local)
for i in range (2,10):
  tuple_list = t_corpus.row_values(i)
  print (tuple_list)
  t_ = df2.loc[(df2['uid']==tuple_list[0]) & (df2['language']==tuple_list[6])]
  if len(t_)==1:
    t_corpus.update_cell(i, 9, t_["hash"].values[0])



['f9bc13ef', 'Médicament', '3', 'À quel moment de la journée vous prenez le médicament?', 'M', '2022-04-01 8:15:21', 'ams', '-7.18338E+18', '-7.18E+18', 'Arabe']
['6e65dde7', 'Abcès', '1', 'Abcès', 'A', '2022-03-29 15:58:04', 'ams', '6.7546E+18', '1.21E+18', 'Arabe']
['80cbacea', 'Abdomen', '1', 'Abdomen', 'A', '2022-03-31 15:25:12', 'ams', '5.94579E+18', '2.24E+18', 'Arabe']
['95c4a863', 'Fébrile', '3', 'Accès fébrile concomitant chez un autre membre de la famille', 'F', '2022-04-01 8:27:19', 'ams', '4.71432E+18', '-2.67E+18', 'Arabe']
['30579682', 'Accident', '1', 'Accident', 'A', '2022-03-29 15:58:05', 'ams', '3.56971E+18', '3.69E+18', 'Arabe']
['6f214e4c', 'Accident', '2', 'Accident cérébral', 'A', '2022-03-29 16:10:09', 'ams', '-3.65209E+18', '-4.54E+18', 'Arabe']
['b67d8739', 'Accouchement', '1', 'Accouchement', 'A', '2022-03-31 14:33:15', 'ams', '-2.5744E+18', '2.60E+17', 'Arabe']
['d1746bc6', 'Accoucher', '1', 'Accoucher', 'A', '2022-03-31 14:33:23', 'ams', '-1.43587E+18', '-4.

In [87]:
t_= df2.loc[(df2['uid']=='f9bc13ef') & (df2['language']=='ams')]

In [88]:
t_["hash"].values[0]

'-7.183381069095526e+18'

In [ ]:
for i in df_local["uid"]:
  #values_list = t_corpus.row_values(1)
  print (i)

In [41]:
cell_list = t_corpus.range('A2:A7')
for cell in cell_list:
    print (cell.value)

f9bc13ef
6e65dde7
80cbacea
95c4a863
30579682
6f214e4c


In [ ]:
# Update multiple ranges at once
t_corpus.batch_update([{
    'range': 'A2:J2',
    'values': [['A1', 'B1'], ['A2', 'B2']],
}, {
    'range': 'J42:K43',
    'values': [[1, 2], [3, 4]],
}])

In [39]:
df2.head(5)

,uid,maitre,ordre,expression,idx,timestamp,language,translation,hash,language_fulltext
0,f9bc13ef,Médicament,3,À quel moment de la journée vous prenez le méd...,M,2022-04-01 08:15:21,ams,في أي وقت من اليوم تتناول الدواء؟,-7183381069095526137,Arabe
1,6e65dde7,Abcès,1,Abcès,A,2022-03-29 15:58:04,ams,خُرّاج,6754604515239380785,Arabe
2,80cbacea,Abdomen,1,Abdomen,A,2022-03-31 15:25:12,ams,NaN,5945788393346531146,Arabe
3,95c4a863,Fébrile,3,Accès fébrile concomitant chez un autre membre...,F,2022-04-01 08:27:19,ams,نوبة حمى مزامنة عند فرد اخر من الأسرة,4714324893756359155,Arabe
4,30579682,Accident,1,Accident,A,2022-03-29 15:58:05,ams,حادث,3569707237868940092,Arabe


In [33]:
df_local.head(5)

,uid,maitre,ordre,expression,idx,timestamp,language,translation,hash,language_fulltext
0,f9bc13ef,Médicament,3,À quel moment de la journée vous prenez le méd...,M,2022-04-01 08:15:21,ams,في أي وقت من اليوم تتناول الدواء؟,4712174746685360000,Arabe
1,6e65dde7,Abcès,1,Abcès,A,2022-03-29 15:58:04,ams,خُرّاج,1206557397878000000,Arabe
2,80cbacea,Abdomen,1,Abdomen,A,2022-03-31 15:25:12,ams,,2241239221854210000,Arabe
3,95c4a863,Fébrile,3,Accès fébrile concomitant chez un autre membre...,F,2022-04-01 08:27:19,ams,نوبة حمى مزامنة عند فرد اخر من الأسرة,-2669702077897510000,Arabe
4,30579682,Accident,1,Accident,A,2022-03-29 15:58:05,ams,حادث,3691916133042400000,Arabe


In [ ]:
frames = [df2, df_local]
result = pd.concat(frames, keys=["hash"])
result.drop_duplicates(subset=['hash'], inplace=True, keep='last')

In [ ]:
result

uid      maitre  ordre                                         expression idx           timestamp language                                        translation                 hash language_fulltext
hash 0     f9bc13ef  Médicament      3  À quel moment de la journée vous prenez le méd...   M 2022-04-01 08:15:21      ams                  في أي وقت من اليوم تتناول الدواء؟  4712174746685359176             Arabe
     1     6e65dde7       Abcès      1                                              Abcès   A 2022-03-29 15:58:04      ams                                             خُرّاج  1206557397878002961             Arabe
     2     80cbacea     Abdomen      1                                            Abdomen   A 2022-03-31 15:25:12      ams                                                NaN  2241239221854210669             Arabe
     3     95c4a863     Fébrile      3  Accès fébrile concomitant chez un autre membre...   F 2022-04-01 08:27:19      ams              نوبة حمى مزامنة عند فرد اخر من الأسرة -2669702077897508385             Arabe
     4     30579682    Accident      1                                           Accident   A 2022-03-29 15:58:05      ams                                               حادث  3691916133042402454             Arabe
...             ...         ...    ...                                                ...  ..                 ...      ...                                                ...                  ...               ...
     5451  d112f379       Poids      3                        Votre poids est-il stable ?   P 2022-03-31 14:33:52      tig                                                NaN -1979989395804264355          Tigrinya
     5452  2344ba84   Accoucher      3  Vous avez accouché par césarienne ou voie basse ?   A 2022-03-29 16:00:33      tig                             ብኦፕረሽን ወይ ብኖርማር ወሊድኪ ?  1648022124426677827          Tigrinya
     5453  36e66161  Contagieux      3            Y a-t-il des cas contagieux à l'école ?   C 2022-03-29 16:00:33      tig                                 አብ ትምህርቲ ዲዩ ለጊብካ ?     1716975638240588          Tigrinya
     5454  c32f5fd9   Chronique      3  Y a-t-il des maladies chroniques dans votre fa...   C 2022-04-01 08:23:46      tig                      ካብ በተሰብካ ሕዱር ሕማሕ ዘለዎ አለካ ዲዩ ?  4364757552740703028          Tigrinya
     5455  d3dba8e4        Zona      1                                               Zona   Z 2022-03-29 16:00:34      tig  ዞና ዝብሃል ሕማም ኣብ ቖርበት ማለት ብደገ ዝቖስል ብርቱዕ ቓንዛ ዘለዎ ... -8449104065387733587          Tigrinya

[5456 rows x 10 columns]

In [ ]:
df2[(df2['hash'].isin(df_local['hash']) & ~df2['timestamp'].isin(df_local['timestamp']) )]

,uid,maitre,ordre,expression,idx,timestamp,language,translation,hash,language_fulltext
1518,b452192e,Hospitalisation,1,Hospitalisation,H,2022-03-31 14:33:34,ukr,NaN,9636487329058650,Ukrainien
2220,8361e66a,Intra-veineux,1,Intra-veineux,I,2022-03-29 15:59:20,hun,Intravénás,24730161157882600,Hongrois


In [ ]:
set_with_dataframe(t_corpus, result)

END

### Vocabulaire avec groupement :


1.   Index alpha
2.   Mot clé
3.   Ordre



In [ ]:
vk_voc = airtable.get_all(view='Vocabulary', sort=['idx_cor','Maitre','Ordre'])
df_voc = pd.DataFrame.from_records((r['fields'] for r in vk_voc))
df_voc = df_voc[['idx_cor','Maitre','Ordre','Proposition','CATEGRAM','TYPE','Thématique']]

### Traductions avec groupement :


1.   Index alpha
2.   Mot clé
3.   Ordre

In [ ]:
df_voc_trad = airtable.get_all(view='Traductions', sort=['idx_cor','Maitre','Ordre'])
df_voc_trad = pd.DataFrame.from_records((r['fields'] for r in df_voc_trad))
df_voc_trad = df_voc_trad[['idx_cor','Maitre','Ordre','Proposition','CATEGRAM','TYPE','Thématique',
             'eng','ams','tur','rus','ukr','rou','hun','tig','alb','geo','arm',
             'dar','pst','esp','pol']]

In [ ]:
# Corpus_consolidation
uri = "https://docs.google.com/spreadsheets/d/1cSnSnnQunL-I7hpyYkIqbtuGr55unVJ3Qe8URSrjTr8/edit?usp=sharing"

wb = get_CCDB_wb(uri)
db_cc = get_CCDB_data (wb, 0)

nom_onglet = 'Lexique'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df_voc_trad)

Onglet inexistant !


In [ ]:
grouped = df_voc.groupby(['idx_cor','Maitre'])
'''
for name, group in grouped:
  print (name)
  print (group["Proposition"])
'''

'\nfor name, group in grouped:\n  print (name)\n  print (group["Proposition"])\n'

Recherche des UID manquants dans Airtable

In [ ]:
def airtable_get_missing_uids(df):
  return df.loc[pd.isna(df["UID"])]


Recherche dans Airtable par Séquence

In [ ]:
def airtable_get_records_by_sequence(df, sequence):
  return df.loc[df["Séquence"]==sequence]


In [ ]:
#df_vkat.loc[df_vkat["Séquence"]=="1. Présentation + antécédents"]

In [ ]:
airtable_get_missing_uids(df_vkat)

,Proposition,CATEGRAM,DEPENDANCE,idx_cor,TYPE,UID,VOCABULAIRE,Thématique,idx,Ordre,...,Updates,Séquence,MedGen,Statut,Fiche,Phrase Assoc,Définition,Corpus copy,DOMAINE,Exp Assoc
6,~ chirurgicale,[exp],[reco8iFRctnJgMJCl],I,expression,NaN,NaN,[Vaccination],I,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Accouchement,[nm],NaN,A,vocabulaire,NaN,NaN,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Accoucher,[vi],NaN,A,vocabulaire,NaN,NaN,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Appareil ~,[exp],NaN,D,expression,NaN,NaN,[Médecine générale],NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,Assurance,[nf],NaN,A,vocabulaire,NaN,Général,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,Urine,[nf],NaN,U,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,Vaccinal,[adj],NaN,V,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,"Vacciné, -e",[adj],NaN,V,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317,Votre poids a-t-il augmenté ?,[phrase],[recnFsFSoE1DCjYqt],P,phrase,NaN,NaN,[Médecine générale],NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Rceherche des termes présents dans Central, présents dans AIRTBL mais sans UID

In [ ]:
vk_at_trad = airtable.get_all(view='Traductions',sort='Proposition')
df_vk_at_trad = pd.DataFrame.from_records((r['fields'] for r in vk_at_trad))
df_vk_at_trad = df_vk_at_trad[["Proposition","idx_cor","UID"]]
df_vk_at_trad_notna = df_vk_at_trad.loc[pd.notna(df_vk_at_trad["UID"])]
df_vk_at_trad_na = df_vk_at_trad.loc[pd.isna(df_vk_at_trad["UID"])]

### merge côté AT : avec les UID présents 

In [ ]:
df_merged = pd.merge(df_cc_global, df_vk_at_trad_notna, how="right", left_on='uid', right_on='UID')

## On complète avec les traductions

In [ ]:
for index, row in df_merged.iterrows():
  if row["uid"] is not None:
    record = airtable.match('UID', row["uid"])
    aDict = {}
    for i in vk_languages:
      key = i["trigramme"]
      aDict[key] = row[key]
      #print (record['id'] +" " + row["eng"])
    airtable.update(record['id'], aDict)


## On complète avec les UID manquants

In [ ]:
df_vk_at_trad_na

In [ ]:
df_merged = pd.merge(df_cc_global, df_vk_at_trad_na, how="right", left_on='expression', right_on='Proposition')

In [ ]:
df_merged = df_merged.loc[pd.notna(df_merged["uid"])]

In [ ]:
for index, row in df_merged.iterrows():
  if row["uid"] is not None:
    record = airtable.match('Proposition', row["expression"])
    aDict = {}
    key = "UID"
    aDict[key] = row["uid"]
    for i in vk_languages:
      key = i["trigramme"]
      aDict[key] = row[key]
      #print (record['id'] +" " + row["eng"])
    airtable.update(record['id'], aDict)

## On vide toute la base AT dans Central Consolidation

In [ ]:
vk_at = airtable.get_all(view='Admin',sort='Proposition')
df_vkat = pd.DataFrame.from_records((r['fields'] for r in vk_at))

In [ ]:
# Corpus_consolidation
uri = "https://docs.google.com/spreadsheets/d/1cSnSnnQunL-I7hpyYkIqbtuGr55unVJ3Qe8URSrjTr8/edit?usp=sharing"

wb = get_CCDB_wb(uri)
db_cc = get_CCDB_data (wb, 0)

nom_onglet = 'Termes'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df_vkat)



END

In [ ]:
record = airtable.match('UID', '30579682')
fields = {'eng': 'Accident'}
airtable.update(record['id'], fields)

{'createdTime': '2021-12-10T08:47:48.000Z',
 'fields': {'CATEGRAM': ['nm'],
  'Name ID': 'recGaRQGjV5qN3QBN',
  'Proposition': 'Accident',
  'TYPE': 'vocabulaire',
  'Thématique': 'Vaccination',
  'UID': '30579682',
  'VOCABULAIRE': 'Général',
  'eng': 'Accident',
  'idx': 'A',
  'idx_cor': 'A'},
 'id': 'recGaRQGjV5qN3QBN'}

In [ ]:
record

{'createdTime': '2021-12-10T08:47:48.000Z',
 'fields': {'CATEGRAM': ['nm'],
  'Name ID': 'recGaRQGjV5qN3QBN',
  'Proposition': 'Accident',
  'TYPE': 'vocabulaire',
  'Thématique': 'Vaccination',
  'UID': '30579682',
  'VOCABULAIRE': 'Général',
  'idx': 'A',
  'idx_cor': 'A'},
 'id': 'recGaRQGjV5qN3QBN'}

In [ ]:
def airtable_table_get(uri, params, headers):
  airtable_records = []
  run = True
  while run is True:
    response = requests.get(uri, params=params, headers=headers)
    airtable_response = response.json()
    airtable_records += (airtable_response['records'])
    if 'offset' in airtable_response:
      run = True
      params = (('offset', airtable_response['offset']),)
    else:
      run = False
  return airtable_records

# Table Corpus

In [ ]:

url = "https://api.airtable.com/v0/" + base_id + "/" + table_name


### On lit tous les enregistrements, puis on met dans un dataframe

In [ ]:
airtable_records = airtable_table_get(url, params, headers)

airtable_rows = [] 
airtable_index = []
for record in airtable_records:
  airtable_rows.append(record['fields'])
  airtable_index.append(record['id'])
df = pd.DataFrame(airtable_rows, index=airtable_index)
df = df [['Proposition','UID', 'Name ID', 'DEPENDANCE', 'idx', 'idx_cor','Statut','Thématique']]
df.sort_values(by=['Proposition'], ascending=True, inplace=True)

In [ ]:
df.head(10)

,Proposition,UID,Name ID,DEPENDANCE,idx,idx_cor,Statut,Thématique
recKbPHqDTXNPwYK0,A jeun,f397cda1,recKbPHqDTXNPwYK0,NaN,J,J,NaN,Vaccination
rec1qTnfWxjhx6SBl,AME - Aide médicale d'État,f49563a9,rec1qTnfWxjhx6SBl,NaN,A,A,NaN,Vaccination
rec0HgtD22F7TWM3W,Abcès,6e65dde7,rec0HgtD22F7TWM3W,NaN,A,A,NaN,Médecine générale
recGaRQGjV5qN3QBN,Accident,30579682,recGaRQGjV5qN3QBN,NaN,A,A,NaN,Vaccination
rec3MxDq67MC982ng,Accident cérébral,6f214e4c,rec3MxDq67MC982ng,[recGaRQGjV5qN3QBN],A,A,NaN,Vaccination
reciexfGbNKvXzQg8,Alcool,NaN,reciexfGbNKvXzQg8,NaN,A,A,NaN,Médecine générale
recfY4uKkgK8Jp8GO,Allergie,8b195cf4,recfY4uKkgK8Jp8GO,NaN,A,A,NaN,Vaccination
recSKSrXVLhDfMq6w,Allergie alimentaire,f754eebf,recSKSrXVLhDfMq6w,[recfY4uKkgK8Jp8GO],A,A,NaN,Vaccination
rectthHdPchXi1zr9,Allergie médicamenteuse,7256c169,rectthHdPchXi1zr9,"[recfY4uKkgK8Jp8GO, recOivQOmeragygVr]",A,A,NaN,Vaccination
recHNzXfvWWm0Lwp8,Allergie à un médicament,316fa7ab,recHNzXfvWWm0Lwp8,"[recfY4uKkgK8Jp8GO, recOivQOmeragygVr]",A,A,NaN,Vaccination


In [ ]:
def lookup_uid (uid):
  try:
    record = df_cc_fr.loc[df_cc_fr["uid"]==uid]["index"].item()
  except Exception as e:
    list_no_uid.append(uid)
    return uid
  if record is not None:
    return record

In [ ]:
x = df_cc_fr.loc[df_cc_fr["uid"]=="72f0e8db"]["index"].item()

In [ ]:
list_no_uid = [] # uid non présents côté central = mots à créer
df['UID'] = df['UID'].apply(lambda x: make_unique_id() if pd.isna(x) else x)
df.idx_cor = df['UID'].apply(lambda x: lookup_uid(x))

convert the dataframe to a dictionary of dictionaries.

In [ ]:
df.tail()

,Proposition,UID,Name ID,DEPENDANCE,idx,idx_cor,Statut,Thématique
recia1L6nICYg6LJ8,Zona,d3dba8e4,recia1L6nICYg6LJ8,NaN,NaN,Z,NaN,Vaccination
recVvIHdNWf80zvsG,À quelle moment de la journée vous prenez le m...,0f86d1ef,recVvIHdNWf80zvsG,[recOivQOmeragygVr],M,0f86d1ef,Ramona,Médecine générale
recWp0rajm2qKcJuF,Êtes vous vacciné (e) ?,c5df267a,recWp0rajm2qKcJuF,NaN,NaN,Ê,NaN,Médecine générale
recBY5NXGM8UtBcaX,Êtes-vous enceinte ?,7768882c,recBY5NXGM8UtBcaX,NaN,NaN,Ê,NaN,Médecine générale
reczy8Os5Hc3qZ1yA,Œdème,72f0e8db,reczy8Os5Hc3qZ1yA,NaN,NaN,Œ,NaN,Médecine générale


In [ ]:
dict_records = df.tail().to_dict(orient="index")

In [ ]:
for record_id, upload_data in dict_records.items():
  record_url = url + "/" + record_id
  upload_dict = {"records" : [{"fields" : upload_data}]}
  upload_json = json.dumps(upload_dict)
  requests.post(url, data=upload_json, headers=headers)

In [ ]:
headers

{'Authorization': 'Bearer keyrI98TIqu6mbFcf',
 'Content-Type': 'application/json'}

In [ ]:
requests.patch(url, data=upload_json, headers=headers)

<Response [422]>

In [ ]:
upload_json='{"records": [{"fields": {"Proposition": "OEDEME", "id": "reczy8Os5Hc3qZ1yA"}}]}'

In [ ]:
requests.post(record_url, data=upload_json, headers=headers)

<Response [404]>

In [ ]:
def fmt_table(wb, nom_onglet):
  # onglet
  ws = wb.worksheet(nom_onglet)

  # Format de la partie gauche
  fmt = cellFormat(
      backgroundColor=color(0.91, 0.96, 0.93),
      textFormat=textFormat(bold=False, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, 'A:', fmt)

  # format de l'entete
  fmt = cellFormat(
      backgroundColor=color(0.7725,0.8431,0.7922),
      textFormat=textFormat(bold=True, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, '1', fmt)
  
  # format de l'auto wrap
  fmt = cellFormat(
      wrapStrategy='WRAP'
      )
  format_cell_range(ws, 'B:C', fmt)

  # on gèle l'entete et les colonnes de gauche
  set_frozen(ws, rows=1, cols=1)
  set_column_width(ws, 'A:C', 500)


In [ ]:
nom_onglet = 'Liste'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df)
fmt_table(wb,nom_onglet)


In [ ]:
table_name = 'Structure'
url = "https://api.airtable.com/v0/" + base_id + "/" + table_name

In [ ]:
df

,Name,Situation de départ,Notes
recnI80hvydv71fs4,0.Conception,"Plus on possède, plus on est soi-même possédé....","Trouver le plaisir d’écrire. Partir d’idées, d..."
recI2pgkNq2iP52rm,1.Situation initiale,Un jour d'été où il pleut et fait un temps abo...,Une vengeance d'une femme éconduite lors d'un ...
recb2DqhB9GvfI12n,2.Personnage,Un homme (marié ?) qui a donné rendez-vous à u...,Un Dom Juan sûr de son charme et imbu de sa pe...
recNLdU01JpCnvknJ,3.Acte déclencheur,La femme lui propose un jeu. Elle est dans le ...,NaN
recRxZVhpSPD6oQXx,4.Réaction,L'homme est d'abord tenté de partir car il est...,NaN
recRr06fOM4hGGaLY,5.Péripéties,"Pendant la visite (ennuyeuse), l'homme tente d...",NaN
recvWnAYn9VNySVlB,6.Chute,Un spectacle où une partie du public est appel...,NaN
recDmZtMMNmiy7R0M,Récit,NaN,NaN
